## 02. 원본 자유필사 이미지에서 단어 자르기

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os, json, sys, re

In [13]:
label_file_list = os.listdir('./Paper_Label/female') + os.listdir('./Paper_Label/male')
len(label_file_list)

12797

In [14]:
def convertSex(sex) :
    if (sex == 0) :
        return 'female'
    else :
        return 'male'

In [15]:
def convertPenType(penType) :
    if (penType == 0) :
        return 'ball'
    elif (penType == 1) :
        return 'marker'
    elif (penType == 2) :
        return 'stylus'
    else :
        return 'etc'

In [16]:
def convertSex(sex) :
    if (sex == 0) :
        return 'female'
    else :
        return 'male'

In [17]:
def convertAgeRange(age) :
    if (10 <= age < 20) :
        return '10s'
    elif (20 <= age < 30) :
        return '20s'
    elif (30 <= age < 40) :
        return '30s'
    elif (40 <= age < 50) :
        return '40s'
    elif (50 <= age < 60) :
        return '50s'
    else :
        return '60s over'

### DO : 이미지 파일에서 단어 잘라서, "파일 이름" + "_인덱스" 형식으로 저장하기 & 라벨링 데이터로 데이터 프레임에 성별, 나이, 단어 개수, 원본 이미지 파일 이름 저장하기.

한글 단어 판별 함수

In [18]:
def isHangul(text):
    #Check the Python Version
    pyVer3 =  sys.version_info >= (3, 0)

    if pyVer3 : # for Ver 3 or later
        encText = text
    else: # for Ver 2.x
        if type(text) is not unicode:
            encText = text.decode('utf-8')
        else:
            encText = text

    hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
    return hanCount > 0

In [19]:
from PIL import Image

label_file_list = os.listdir('./Paper_Label/female') + os.listdir('./Paper_Label/male')
len(label_file_list)

word_dict_list = []
file_location = ['./Paper_Label/female/', './Paper_Label/male/']
image_location = './Paper_Png/'

for location in file_location :
    for label_file in os.listdir(location):
        with open(location+label_file, 'r', encoding='utf8') as f:
            info_json = json.load(f)
        img = Image.open(image_location + label_file[:-4] + 'png')
        writer_sex = info_json['Images']['writer_sex']
        writer_age = info_json['Images']['writer_age']
        writer_age_range = convertAgeRange(writer_age)

        for cnt, word in enumerate(info_json['bbox']):
            if isHangul(word['data']) and 2<=len(word['data'])<=4:
                word_len = len(word['data'])
                #이미지 crop 해서 ./Word_Cut_Png에 저장
                # 이때 파일명은 원래 파일명 + 저장되는 순서로 한다.
                
                cut_img = img.crop((word['x'][0], word['y'][0], word['x'][3], word['y'][3]))
                cut_img.save('./Word_Cut_Png/' + label_file[:-5] + '_' + str(cnt) + '.png')
                word_dict_list.append({
                    'image_file_name': label_file[:-5] + '_' + str(cnt) + '.png',
                    'sex' : convertSex(writer_sex),
                    'age' : writer_age,
                    'age_range' : writer_age_range,
                    'word' : word['data'],
                    'word_len' : word_len
                })

df = pd.DataFrame(word_dict_list)

In [20]:
df.head()

,image_file_name,sex,age,age_range,word,word_len
0,IMG_OCR_53_4PR_09305_1.png,female,41,40s,강원도,3
1,IMG_OCR_53_4PR_09305_2.png,female,41,40s,경상북도,4
2,IMG_OCR_53_4PR_09305_3.png,female,41,40s,경기도,3
3,IMG_OCR_53_4PR_09305_4.png,female,41,40s,인제군,3
4,IMG_OCR_53_4PR_09305_5.png,female,41,40s,영광군,3


In [22]:
df.to_csv('word_data.csv', encoding = 'utf-8', index = False)

In [23]:
df2 = pd.read_csv('word_data.csv')
df2.head()

,image_file_name,sex,age,age_range,word,word_len
0,IMG_OCR_53_4PR_09305_1.png,female,41,40s,강원도,3
1,IMG_OCR_53_4PR_09305_2.png,female,41,40s,경상북도,4
2,IMG_OCR_53_4PR_09305_3.png,female,41,40s,경기도,3
3,IMG_OCR_53_4PR_09305_4.png,female,41,40s,인제군,3
4,IMG_OCR_53_4PR_09305_5.png,female,41,40s,영광군,3
